In [19]:
import pulp
import pandas as pd
import random

In [20]:


# Sample data for SKUs. Each dictionary represents a product with its attributes.
# In practice, you could load this from a CSV or database.
data = [
    {"sku": "sku1", "net_sales": 100, "OTIF": 0.95, "turnover": 200, "NPS": 8, "family": "A"},
    {"sku": "sku2", "net_sales": 150, "OTIF": 0.90, "turnover": 300, "NPS": 7, "family": "B"},
    {"sku": "sku3", "net_sales": 120, "OTIF": 0.98, "turnover": 250, "NPS": 9, "family": "C"},
    {"sku": "sku4", "net_sales": 130, "OTIF": 0.92, "turnover": 220, "NPS": 8, "family": "D"},
    # Add additional SKU records as needed
]

In [21]:
pd.DataFrame(data)

,sku,net_sales,OTIF,turnover,NPS,family
0,sku1,100,0.95,200,8,A
1,sku2,150,0.90,300,7,B
2,sku3,120,0.98,250,9,C
3,sku4,130,0.92,220,8,D


In [26]:
otif_gen = lambda : round(random.random() * 100, 2)
sku_gen = lambda : 'sku' + str(random.randint(0, 100))
int_gen = lambda x: random.randint(0, x)
float_gen = lambda x: random.random() * x
family_gen = lambda : random.choice(['A', 'B', 'C', 'D', 'E'])

data = [
    {
        "sku": 'sku' + str(i),
        "net_sales": float_gen(10000),
        "OTIF": otif_gen(),
        "turnover": int_gen(100),
        "NPS": int_gen(100),
        "family": int_gen(100)
    } for i in range(100000)
]

pd.DataFrame(data)

,sku,net_sales,OTIF,turnover,NPS,family
0,sku0,6860.686684,83.00,72,63,78
1,sku1,8940.744122,22.91,8,88,75
2,sku2,4070.158364,43.97,78,44,1
3,sku3,6000.697971,28.52,35,54,88
4,sku4,9806.933371,39.70,24,95,46
...,...,...,...,...,...,...
99995,sku99995,6425.565675,44.83,26,66,72
99996,sku99996,6364.821265,71.62,9,5,68
99997,sku99997,570.812081,72.48,5,47,15
99998,sku99998,7063.611250,10.63,57,15,44


In [27]:
# Define parameters
max_skus = 100          # Maximum number of SKUs that can be placed at the checkout shelf.
OTIF_min = 0.4          # Minimum average OTIF required.
turnover_min = 40       # Minimum total turnover required.
NPS_min = 7             # Minimum average NPS required.

# Create the optimization problem (maximization)
prob = pulp.LpProblem("Retail_Shelf_Optimization", pulp.LpMaximize)

# Define decision variables: x_i = 1 if SKU i is selected; 0 otherwise.
sku_vars = {item["sku"]: pulp.LpVariable(f"x_{item['sku']}", cat='Binary') for item in data}

# Objective: Maximize total net sales from selected SKUs.
prob += pulp.lpSum(item["net_sales"] * sku_vars[item["sku"]] for item in data), "Total_Net_Sales"

# Constraint 1: Select at most 'max_skus' SKUs.
prob += pulp.lpSum(sku_vars[item["sku"]] for item in data) <= max_skus, "Max_SKUs_Constraint"

# Constraint 2: Ensure the average OTIF of selected SKUs is at least OTIF_min.
# This can be written as: sum((OTIF_i - OTIF_min)*x_i) >= 0
prob += pulp.lpSum((item["OTIF"] - OTIF_min) * sku_vars[item["sku"]] for item in data) >= 0, "OTIF_Constraint"

# Constraint 3: Do not select more than one SKU per product family.
families = {}
for item in data:
    fam = item["family"]
    families.setdefault(fam, []).append(item["sku"])

for fam, sku_list in families.items():
    prob += pulp.lpSum(sku_vars[sku] for sku in sku_list) <= 1, f"Family_Constraint_{fam}"

# Constraint 4: Ensure the total turnover from selected SKUs is at least turnover_min.
prob += pulp.lpSum(item["turnover"] * sku_vars[item["sku"]] for item in data) >= turnover_min, "Turnover_Constraint"

# Constraint 5: Ensure the average NPS of selected SKUs is at least NPS_min.
# Similarly, express as: sum((NPS_i - NPS_min)*x_i) >= 0
prob += pulp.lpSum((item["NPS"] - NPS_min) * sku_vars[item["sku"]] for item in data) >= 0, "NPS_Constraint"

# Solve the problem using the default CBC solver (free and offline)
solver = pulp.PULP_CBC_CMD()
result_status = prob.solve(solver)

# Output the results
print("Status:", pulp.LpStatus[prob.status])
selected_skus = [item["sku"] for item in data if pulp.value(sku_vars[item["sku"]]) == 1]
print("Selected SKUs:", selected_skus)
print("Total Net Sales:", pulp.value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/fabioyamada/miniconda3/envs/.optimization/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/1j/2s04pnsd0_n4srzj9y2v33qh0000gn/T/516d30fd77e14a4383959162a71faf79-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/1j/2s04pnsd0_n4srzj9y2v33qh0000gn/T/516d30fd77e14a4383959162a71faf79-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 110 COLUMNS
At line 798153 RHS
At line 798259 BOUNDS
At line 898260 ENDATA
Problem MODEL has 105 rows, 100000 columns and 498042 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 998986 - 0.27 seconds
Cgl0008I 101 inequality constraints converted to equality constraints
Cgl0005I 101 SOS with 95255 members
Cgl0004I processed model has 105 rows, 95255 columns (95255 integer (95255 of which binary)) and 473968 e